In [1]:
import pandas as pd
import sqlite3
import datetime

#read csv into a pandas dataframe
reviews_df = pd.read_csv("../reddit_exercise_data.csv")

#quick eye balling!
reviews_df.head()

,title,review,product_name,iso,score,date,app_bought,money_spent
0,Love it!,I use this app to read r/nosleep stories befor...,Reddit Official App: Trending News and Hot Topics,US,5,17-7-10 0:0:0,47,140
1,•••,Dank memes,Reddit Official App: Trending News and Hot Topics,MY,5,17-7-10 0:0:0,3,10
2,Very good app,"Love the app, makes using Reddit very convenient",Reddit Official App: Trending News and Hot Topics,US,5,17-7-10 0:0:0,61,182
3,CNN here,As the CEO of CNN I find this app repulsive. A...,Reddit Official App: Trending News and Hot Topics,US,5,17-7-10 0:0:0,3,6
4,Easy peasy!,"I was infrequent in my visits, mainly mobile w...",Reddit Official App: Trending News and Hot Topics,US,5,17-7-10 0:0:0,59,178


In [2]:
reviews_df.dtypes

title           object
review          object
product_name    object
iso             object
score            int64
date            object
app_bought       int64
money_spent      int64
dtype: object

# Cleaning-up Data

<b>Objectives- </b>
1. Make sure there are no duplicate entries. Because users can accidentally post same comments twice.
2. Need to check ISOs are valid - since they are limitied few, can do a quick eyeball at unique values
3. Fix date format - it should be consistent for all rows
4. Make sure that score, app_bought and money_spent are numeric values for all rows - can be confirmed by data types
5. Review, title and product name: not required to clean up in my opinion for the purpose of this analysis

In [3]:
#drop duplicates inplace
reviews_df.drop_duplicates(keep=False,inplace=True)

In [4]:
#check iso values
reviews_df.iso.unique()

array(['US', 'MY', 'CA', 'DE', 'CN', 'CZ', 'NL', 'CH', 'PL', 'RO', 'AT',
       'ZA', 'CO', 'RU', 'BE', 'MX', 'AU', 'TR', 'IN', 'DK', 'HU', 'SG',
       'HR', 'GB', 'NZ', 'NO', 'SE', 'TW', 'GR', 'IT', 'PH', 'TH', 'BR',
       'EG', 'KH', 'CR', 'KR', 'JP', 'FR', 'ES', 'SK', 'IE', 'HK', 'FI',
       'AR', 'JO', 'IS', 'UA', 'BG', 'EC', 'CY', 'CL', 'PT', 'MT', 'LK',
       'OM', 'BY', 'GT', 'NG', 'IL', 'VN', 'PE', 'BH', 'TZ', 'MK', 'LV',
       'TN', 'AE', 'PK', 'AL', 'KZ', 'SI', 'HN', 'KW', 'BN', 'ID', 'LB',
       'UY', 'DO', 'NE', 'LT'], dtype=object)

In [5]:
#format date column of original dataframe to DD-MM-YYYY format
def formatDate(input_date):
    idate = input_date.split(" ")[0]
    if idate.find('/') >= 0:
        month,date,year = [int(x) for x in idate.split('/')]
    else:
        year,month,date = [int(x) for x in idate.split('-')]
    year = 2000+year if year < 2000 else year
    return datetime.date(year,month,date).strftime("%d-%m-%Y")

In [6]:
reviews_df.date = reviews_df.date.apply(formatDate)

# Deciding Bucket Size

In [7]:
#Parameters for decision on bucket size for app_bought
#Interested to see how is data distributed
(reviews_df.app_bought.min(), reviews_df.app_bought.max(), reviews_df.app_bought.mean(), reviews_df.app_bought.median())

(0, 100, 48.7212, 48.0)

In [8]:
#Median and Average are close to actual mid of range, choosing bucket size of 10 for app_bought
def getAppBoughtBucket(app_bought):
    return str(app_bought//10*10)+"-"+str((app_bought//10+1)*10)

reviews_df['app_bought_bucket'] = reviews_df.app_bought.apply(getAppBoughtBucket)

In [9]:
#Parameters for decision on bucket size for money_spent
(reviews_df.money_spent.min(), reviews_df.money_spent.max(), reviews_df.money_spent.mean(), reviews_df.money_spent.median())

(0, 500, 143.4616, 108.0)

In [10]:
# the centrality analysis reveales that despite a large range of 500, most data points lie in 
# the lower range. Choosing small bucket size of 25 in interest of more granular information.

reviews_df['money_spent_bucket'] = reviews_df.money_spent.apply(lambda x:str(x//25*25)+"-"+str((x//25+1)*25))

In [11]:
reviews_df.head()

,title,review,product_name,iso,score,date,app_bought,money_spent,app_bought_bucket,money_spent_bucket
0,Love it!,I use this app to read r/nosleep stories befor...,Reddit Official App: Trending News and Hot Topics,US,5,10-07-2017,47,140,40-50,125-150
1,•••,Dank memes,Reddit Official App: Trending News and Hot Topics,MY,5,10-07-2017,3,10,0-10,0-25
2,Very good app,"Love the app, makes using Reddit very convenient",Reddit Official App: Trending News and Hot Topics,US,5,10-07-2017,61,182,60-70,175-200
3,CNN here,As the CEO of CNN I find this app repulsive. A...,Reddit Official App: Trending News and Hot Topics,US,5,10-07-2017,3,6,0-10,0-25
4,Easy peasy!,"I was infrequent in my visits, mainly mobile w...",Reddit Official App: Trending News and Hot Topics,US,5,10-07-2017,59,178,50-60,175-200


# Writing cleaned up data to csv file

In [12]:
#cleaned up csv
reviews_df[['title', 'review', 'product_name', 'iso', 'score', 'date', 'app_bought', 'money_spent']].to_csv('reviews.csv')

# Writing to sqlite database engine

In [13]:
#persist into sqlite database
conn = sqlite3.connect('exercise_database.db')
df = reviews_df[['review','title','iso','score','date','app_bought','money_spent','app_bought_bucket','money_spent_bucket']]
df.to_sql('reviews', conn, index=False, if_exists='replace')
conn.close()

# Querying sqlite database

In [14]:
#read from sqlite database and perform queries
conn = sqlite3.connect("exercise_database.db")
query1 = """
    SELECT 
        iso, AVG(score) AS avg_score
    FROM reviews 
    GROUP BY iso """
query2 = """
    SELECT
        app_bought_bucket, MAX(score) AS max_score
    FROM reviews
    GROUP BY app_bought_bucket """
query3 = """
    SELECT
        date AS day, AVG(score) AS avg_score
    FROM reviews
    GROUP BY date """


In [15]:
pd.read_sql_query(query1, conn).head()

,iso,avg_score
0,AE,4.714286
1,AL,5.000000
2,AR,4.900000
3,AT,4.461538
4,AU,4.505618


In [16]:
pd.read_sql_query(query2, conn)

,app_bought_bucket,max_score
0,0-10,5
1,10-20,5
2,100-110,5
3,20-30,5
4,30-40,5
5,40-50,5
6,50-60,5
7,60-70,5
8,70-80,5
9,80-90,5


In [17]:
pd.read_sql_query(query3, conn).head()

,day,avg_score
0,01-06-2017,4.516624
1,01-07-2017,4.718750
2,02-06-2017,4.523605
3,02-07-2017,4.689655
4,03-06-2017,4.670732


In [18]:
#closing connection
conn.close()